In [297]:
#Mughees
import pandas as pd 
import plotly.express as px
import nbformat
from datetime import datetime

In [298]:
MartysData = pd.read_excel("Martys_Traffic_counts_Fall_23_to_Spring_24.xlsx", sheet_name='Cleaning Data')


In [299]:
MartysData = MartysData.drop(columns=['Total'])
MartysData

,Date,Day,Start,End,MartyCount
0,2023-09-03,Sunday,12:46:44.320000,13:00:00,4
1,2023-09-03,Sunday,14:46:48.005000,15:00:00,3
2,2023-09-03,Sunday,15:01:52.465000,15:15:00,7
3,2023-09-03,Sunday,15:16:05.633000,15:30:00,6
4,2023-09-03,Sunday,15:31:20.045000,15:45:00,5
...,...,...,...,...,...
4595,2024-05-15,Wednesday,20:16:02.428000,20:30:00,1
4596,2024-05-15,Wednesday,20:31:02.189000,20:45:00,2
4597,2024-05-15,Wednesday,20:46:13.250000,21:00:00,1
4598,2024-05-15,Wednesday,21:16:19.793000,21:30:00,2


In [300]:
# specifying the column names that I want to use
new_cols_dict ={
    'MartyCount':'Customer',}

MartysData.rename(new_cols_dict, axis=1, inplace=True)

In [301]:
MartysData.describe()

,Date,Customer
count,4600,4600.000000
mean,2024-01-11 20:40:16.695652096,6.579565
min,2023-09-03 00:00:00,1.000000
25%,2023-10-29 00:00:00,3.000000
50%,2024-01-24 00:00:00,5.000000
75%,2024-03-27 00:00:00,9.000000
max,2024-05-15 00:00:00,35.000000
std,NaN,5.088794


In [302]:
databyday = MartysData.groupby(['Day'])['Customer'].sum().reset_index

In [303]:
databyday

<bound method Series.reset_index of Day
Friday       1312
Monday       6088
Saturday       63
Sunday       6349
Thursday     5163
Tuesday      5665
Wednesday    5626
Name: Customer, dtype: int64>

In [304]:
print(MartysData['Start'].head())
print(MartysData['Start'].dtype)

0    12:46:44.320000
1    14:46:48.005000
2    15:01:52.465000
3    15:16:05.633000
4    15:31:20.045000
Name: Start, dtype: object
object


In [305]:
# Function to convert strings to time if they are not already in datetime.time format
def convert_to_time(x):
    if isinstance(x, str):  # If the value is a string
        return datetime.strptime(x, '%H:%M:%S.%f').time()
    return x  # If it's already a datetime.time object, return it as is

# Apply the conversion function
MartysData['Start'] = MartysData['Start'].apply(convert_to_time)

In [306]:
print(MartysData['Start'].head())
print(MartysData['Start'].dtype)

0    12:46:44.320000
1    14:46:48.005000
2    15:01:52.465000
3    15:16:05.633000
4    15:31:20.045000
Name: Start, dtype: object
object


In [307]:
# Convert 'Date' to datetime if not already in datetime format
MartysData['Date'] = pd.to_datetime(MartysData['Date'])

# Combine 'Date' and 'Start' to create a full datetime
MartysData['Start'] = MartysData.apply(lambda row: datetime.combine(row['Date'], row['Start']), axis=1)

# Extract the hour from 'Start'
MartysData['Hour'] = MartysData['Start'].dt.hour

# MartysData['Month'] = MartysData['Start'].dt.month

MartysData['Month'] = MartysData['Start'].dt.strftime('%B')


In [308]:
MartysData

,Date,Day,Start,End,Customer,Hour,Month
0,2023-09-03,Sunday,2023-09-03 12:46:44.320,13:00:00,4,12,September
1,2023-09-03,Sunday,2023-09-03 14:46:48.005,15:00:00,3,14,September
2,2023-09-03,Sunday,2023-09-03 15:01:52.465,15:15:00,7,15,September
3,2023-09-03,Sunday,2023-09-03 15:16:05.633,15:30:00,6,15,September
4,2023-09-03,Sunday,2023-09-03 15:31:20.045,15:45:00,5,15,September
...,...,...,...,...,...,...,...
4595,2024-05-15,Wednesday,2024-05-15 20:16:02.428,20:30:00,1,20,May
4596,2024-05-15,Wednesday,2024-05-15 20:31:02.189,20:45:00,2,20,May
4597,2024-05-15,Wednesday,2024-05-15 20:46:13.250,21:00:00,1,20,May
4598,2024-05-15,Wednesday,2024-05-15 21:16:19.793,21:30:00,2,21,May


In [309]:
customer_count_per_hour = MartysData.groupby('Hour')['Customer'].sum()


In [310]:
customer_count_per_hour

Hour
7        1
9        3
10       3
11      26
12      46
13      26
14      69
15    1228
16    1925
17    5920
18    6585
19    5008
20    3999
21    3627
22    1796
23       4
Name: Customer, dtype: int64

In [312]:
# Convert the Series to a DataFrame for compatibility with Plotly
customer_count_per_hour = customer_count_per_hour.reset_index()

filtered_data = customer_count_per_hour[customer_count_per_hour['Hour'] > 14]

# Create the pie chart
fig = px.pie(filtered_data, 
             values='Customer', 
             names='Hour',
             title='Number of Customers per Hour',
             labels={'Customer': 'Number of Customers', 'Hour': 'Hour of the Day'})

# Show the plot
fig.show()


In [283]:
customer_count_per_day= MartysData.groupby(['Day', 'Hour'])['Customer'].sum()


In [284]:
customer_count_per_day

Day        Hour
Friday     12         1
           13         2
           14         3
           15        71
           16        72
                   ... 
Wednesday  19      1013
           20       705
           21       770
           22       433
           23         1
Name: Customer, Length: 83, dtype: int64

In [285]:
def get_global_max_for_days(dataframe, days):
    """
    Calculate the global maximum number of customers for setting a consistent y-axis scale.
    """
    max_customers = 0
    for day in days:
        dept_data = dataframe.loc[day].reset_index()
        all_hours = pd.DataFrame({'Hour': range(0, 24)})  # Ensure all hours are included
        dept_data = pd.merge(all_hours, dept_data, on='Hour', how='left')
        dept_data['Customer'] = dept_data['Customer'].fillna(0)
        max_customers = max(max_customers, dept_data['Customer'].max())
    return max_customers

def plot_Customers_by_Day(dataframe, day, global_max):
    """
    Plot the number of customers by hour for a specific day.
    """
    # Filter the data for the specified day
    dept_data = dataframe.loc[day].reset_index()
    
    # Ensure that all hours (0 to 23) are present
    all_hours = pd.DataFrame({'Hour': range(14, 24)})
    dept_data = pd.merge(all_hours, dept_data, on='Hour', how='left')
    dept_data['Customer'] = dept_data['Customer'].fillna(0)
    
    # Create the line plot with values displayed on the line
    fig = px.line(dept_data, x='Hour', y='Customer', text='Customer',
                  title=f'{day} Customers',
                  labels={'Customer': 'Number of Customers', 'Hour': 'Hour of the Day'},
                  markers=True)
    
    fig.update_traces(textposition='top center')  # Adjust the text position as needed
    fig.update_layout(yaxis=dict(range=[0, global_max + 200]))  # Use the global max for consistent y-axis scale
    fig.update_xaxes(dtick=1)  # Ensure that all hours are shown on the x-axis
    
    fig.show()

In [286]:
UniqueDays = MartysData['Day'].unique()
global_max = get_global_max_for_days(customer_count_per_day, UniqueDays)

UniqueDays
global_max

1587.0

In [287]:
for day in UniqueDays:
    plot_Customers_by_Day(customer_count_per_day, day, global_max)


In [288]:
customer_count_per_month = MartysData.groupby(['Month', 'Hour'])['Customer'].sum()


In [289]:
customer_count_per_month

Month      Hour
April      11        2
           12        1
           13        7
           14        7
           15      251
                  ... 
September  18      988
           19      915
           20      515
           21      548
           22      325
Name: Customer, Length: 108, dtype: int64

In [290]:
def get_global_max(dataframe, months):
    """
    Calculate the global maximum number of customers for setting a consistent y-axis scale.
    """
    max_customers = 0
    for month in months:
        monthly_data = dataframe.loc[month].reset_index()
        all_hours = pd.DataFrame({'Hour': range(0, 24)})
        monthly_data = pd.merge(all_hours, monthly_data, on='Hour', how='left')
        monthly_data['Customer'] = monthly_data['Customer'].fillna(0)
        max_customers = max(max_customers, monthly_data['Customer'].max())
    return max_customers

def plot_Customers_by_Month(dataframe, month, global_max):
    """
    Plot the number of customers by hour for a specific month.
    """
    # Filter the data for the specified month
    monthly_data = dataframe.loc[month].reset_index()
    
    # Ensure that all hours (0 to 23) are present
    all_hours = pd.DataFrame({'Hour': range(14, 24)})
    monthly_data = pd.merge(all_hours, monthly_data, on='Hour', how='left')
    monthly_data['Customer'] = monthly_data['Customer'].fillna(0)
    
    # Create the line plot
    fig = px.line(monthly_data, x='Hour', y='Customer', text='Customer',
                  title=f'{month} Customers',
                  labels={'Customer': 'Number of Customers', 'Hour': 'Hour of the Day'},
                  markers=True)
    
    fig.update_traces(textposition='top center')  # Adjust the text position as needed
    fig.update_layout(yaxis=dict(range=[0, global_max + 200]))  # Use the global max for consistent y-axis scale
    fig.update_xaxes(dtick=1)  # Ensure that all hours are shown on the x-axis
    
    fig.show()

In [291]:
UniqueMonths = MartysData['Month'].unique()
global_max = get_global_max(customer_count_per_month, UniqueMonths)
UniqueMonths
global_max

1240.0

In [292]:
for month in UniqueMonths:
    plot_Customers_by_Month(customer_count_per_month, month, global_max)
